## Frameshift finder
### Import fasta file
### Translate file need to use a single translation table. Need to group fasta input by tax groups if using alternative translation tables
### Create initial alignments of DNA and protein sequences
### Scan the DNA alignment for gaps < 3 bp. If gap is found, these are reported along with the location. The gap then converted N, the sequence is retranslated and relaligned to determine if the read frame can be restored. 
### Need to consider insertion frameshifts as well as deletions. 

In [12]:
import Bio
import pandas as pd 
import os
import subprocess
import re
import sys

### Read input file and translate

In [7]:
# enter files names and variables
input_file = "RecAmod.fsa"
protein_trans = "RecA.pro"
translation_table = "11"
nuc_alignment = "RecA.aln"
prot_alignment = "RecA_pro.aln"
nuc_align_fsa = "RecA.alnfsa"
outputfile = "revisedfasta.fsa"
outputfile_aln = "revisedfasta.aln"
outputfile_alnfsa = "revisedfasta.alnfsa"
revised_pro_aln = "revisedprotein.aln"

In [3]:
from Bio import SeqIO
coding_dna = []
protein_seq = []
trans = []
translations = []
for seq_record in SeqIO.parse(input_file, "fasta"):
    s = seq_record
    coding_dna = s.seq
    s.seq = s.seq.translate(table = translation_table, to_stop=True)
    protein_seq.append(s)
#print(protein_seq)
SeqIO.write(protein_seq, protein_trans, "fasta")

14

### Create alignments

In [4]:
from Bio import AlignIO
help(AlignIO)

Help on package Bio.AlignIO in Bio:

NAME
    Bio.AlignIO - Multiple sequence alignment input/output as alignment objects.

DESCRIPTION
    The Bio.AlignIO interface is deliberately very similar to Bio.SeqIO, and in
    fact the two are connected internally.  Both modules use the same set of file
    format names (lower case strings).  From the user's perspective, you can read
    in a PHYLIP file containing one or more alignments using Bio.AlignIO, or you
    can read in the sequences within these alignments using Bio.SeqIO.
    
    Bio.AlignIO is also documented at http://biopython.org/wiki/AlignIO and by
    a whole chapter in our tutorial:
    
    * `HTML Tutorial`_
    * `PDF Tutorial`_
    
    .. _`HTML Tutorial`: http://biopython.org/DIST/docs/tutorial/Tutorial.html
    .. _`PDF Tutorial`: http://biopython.org/DIST/docs/tutorial/Tutorial.pdf
    
    Input
    -----
    For the typical special case when your file or handle contains one and only
    one alignment, use the func

### Option to read an alignment prepared with another tool and saved locally

In [288]:
#from Bio import AlignIO

#alignment = AlignIO.read(open("align.out"), "fasta")
#print("Alignment length %i" % alignment.get_alignment_length())
#for record in alignment:
#    print(record.seq + " " + record.id)

## Create clustal alignments

In [5]:
from Bio.Align.Applications import ClustalwCommandline
help(ClustalwCommandline)

Help on class ClustalwCommandline in module Bio.Align.Applications._Clustalw:

class ClustalwCommandline(Bio.Application.AbstractCommandline)
 |  ClustalwCommandline(cmd='clustalw', **kwargs)
 |  
 |  Command line wrapper for clustalw (version one or two).
 |  
 |  http://www.clustal.org/
 |  
 |  Notes
 |  -----
 |  Last checked against versions: 1.83 and 2.1
 |  
 |  References
 |  ----------
 |  Larkin MA, Blackshields G, Brown NP, Chenna R, McGettigan PA,
 |  McWilliam H, Valentin F, Wallace IM, Wilm A, Lopez R, Thompson JD,
 |  Gibson TJ, Higgins DG. (2007). Clustal W and Clustal X version 2.0.
 |  Bioinformatics, 23, 2947-2948.
 |  
 |  Examples
 |  --------
 |  >>> from Bio.Align.Applications import ClustalwCommandline
 |  >>> in_file = "unaligned.fasta"
 |  >>> clustalw_cline = ClustalwCommandline("clustalw2", infile=in_file)
 |  >>> print(clustalw_cline)
 |  clustalw2 -infile=unaligned.fasta
 |  
 |  You would typically run the command line with clustalw_cline() or via
 |  the

In [6]:
from Bio import AlignIO
from Bio.Align.Applications import ClustalwCommandline
in_file = input_file
out_file = nuc_alignment
clustalw_cline = ClustalwCommandline("clustalo", infile=in_file, outfile=out_file)
print(clustalw_cline)


clustalo -infile=RecAmod.fsa -outfile=RecA.aln


In [7]:
clustalw_cline()

('Using 8 threads\nRead 14 sequences (type: DNA) from RecAmod.fsa\nnot more sequences (14) than cluster-size (100), turn off mBed\nCalculating pairwise ktuple-distances...\nKtuple-distance calculation progress: 0 % (0 out of 105)\nKtuple-distance calculation progress: 1 % (2 out of 105)\nKtuple-distance calculation progress: 2 % (3 out of 105)\nKtuple-distance calculation progress: 3 % (4 out of 105)\nKtuple-distance calculation progress: 4 % (5 out of 105)\nKtuple-distance calculation progress: 5 % (6 out of 105)\nKtuple-distance calculation progress: 40 % (42 out of 105)\nKtuple-distance calculation progress: 55 % (58 out of 105)\nKtuple-distance calculation progress: 58 % (61 out of 105)\nKtuple-distance calculation progress: 68 % (72 out of 105)\nKtuple-distance calculation progress: 70 % (74 out of 105)\nKtuple-distance calculation progress: 73 % (77 out of 105)\nKtuple-distance calculation progress done. CPU time: 0.10u 0.00s 00:00:00.10 Elapsed: 00:00:00\nGuide-tree computation 

In [8]:
alignment2 = AlignIO.read(open(nuc_alignment), "clustal")
print("Alignment length %i" % alignment2.get_alignment_length())
for record in alignment2:
    print(record.seq + " " + record.id)

Alignment length 1071
ATGCAAGACATCATCCAA----CACTTA-----------------AAAAGCCAGCATCTTGTTTGGCAAGCAAATCTAACCAAAGCTGAC---------------GATCATCAGTTCCTTAATAGTTCTGGTTTTCCGGAGCTAGATGCTCTGCTTGGTGGCGGCTTTCCTCCTCACGGTGTGGTGGAAATGGAATCCATAGGCAGCATTGG-CGAGCTGC---------------GTTTGCTTGCGCCTTACTTAAAAGCATCCCTTT-------------CGAGAG--------------GAGTCACGGCATTCATTCAGCCACCAGCATTGATGAATTCATTGTTCCTACACGGCATTGGGCTAGACATCAACCAGGT----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------GTGGGTGGTCACGC---------------CTGAACATCAGCGCGATGCATTATGGGCAGCGGAG-----CAGTGCCTCAAGAGCGGTGTTTGTACCAACGTTTT------------------GCTGTGGCA--------AGATGAGCTAGAGATCCACCAAGTAAAACGTCTGCAAGTGGCGAGCGAGCAAGGATCTTGCCCTCTGT--TTATGCTCAAACCTAATATGACAAACCGCTTGTCTTTACCTGTTTCG-------------TTGAGTCTGAAATTGCAAGGTCATGGACAAGGCGTTAGCGTGGAAGTTCTCAAACGAAAAGGCGGTTGGAGCAAAGGTAGTGTCGA-CATCAATTTCCAACACCACTACCCACAACTGGTGATGCCAACCACGACTCACATTCCATCAAC

In [9]:
from Bio.Align.Applications import ClustalwCommandline
in_file = protein_trans
out_file = prot_alignment
clustalw_cline = ClustalwCommandline("clustalo", infile=in_file, outfile=out_file)
print(clustalw_cline)

clustalo -infile=RecA.pro -outfile=RecA_pro.aln


In [10]:
clustalw_cline()

('Using 8 threads\nRead 14 sequences (type: Protein) from RecA.pro\nnot more sequences (14) than cluster-size (100), turn off mBed\nCalculating pairwise ktuple-distances...\nKtuple-distance calculation progress: 0 % (0 out of 105)\nKtuple-distance calculation progress: 2 % (3 out of 105)\nKtuple-distance calculation progress: 3 % (4 out of 105)\nKtuple-distance calculation progress: 5 % (6 out of 105)\nKtuple-distance calculation progress: 35 % (37 out of 105)\nKtuple-distance calculation progress: 39 % (41 out of 105)\nKtuple-distance calculation progress: 40 % (43 out of 105)\nKtuple-distance calculation progress: 51 % (54 out of 105)\nKtuple-distance calculation progress: 55 % (58 out of 105)\nKtuple-distance calculation progress: 69 % (73 out of 105)\nKtuple-distance calculation progress: 72 % (76 out of 105)\nKtuple-distance calculation progress: 74 % (78 out of 105)\nKtuple-distance calculation progress done. CPU time: 0.01u 0.00s 00:00:00.01 Elapsed: 00:00:00\nGuide-tree computa

In [9]:
from Bio import AlignIO
prot_alignment = list(AlignIO.read(open("RecA_pro.aln"), "clustal"))
print("Alignment length %i" % prot_alignment.get_alignment_length())
for record in prot_alignment:
    print(record.seq + " " + record.id)

AttributeError: 'list' object has no attribute 'get_prot_alignment_length'

## Convert alignment2 from clustal to fasta

In [10]:
from Bio import SeqIO
alignment3 = AlignIO.read("RecA_pro.aln", "clustal")
#print(alignment3)
proaln_fasta = SeqIO.write(alignment3, "prot_alignment_fsa", "fasta")

In [15]:
print(proaln_fasta)

14


## Score alignments
### https://github.com/benhid/pyMSA/blob/master/resources/tutorial-pymsa.pdf

In [98]:
pip install pyMSA

  Preparing metadata (setup.py) ... done
  Created wheel for pyMSA: filename=pyMSA-0.8.1-py3-none-any.whl size=12824 sha256=a71b85fe50802981c74b4b334beb49f7000a39468af1baa4e08c2d21e2e21e05
  Stored in directory: /Users/rich/Library/Caches/pip/wheels/c7/00/54/1acf56628af0e3684b5ba4395448fa3ca59700ce22994e5935
Successfully built pyMSA
Note: you may need to restart the kernel to use updated packages.


In [7]:
file_name = proaln_fasta
list_of_pairs = []
key = ''
value = ''

with open(file_name, 'r') as file:
    for line in file:
        if line[0] == '>':
            if key != '':
                list_of_pairs.append((key, value))
            key = line[1:].rstrip()
            value = ''
        else:
            value += line.rstrip()

list_of_pairs.append((key, value))

NameError: name 'proaln_fasta' is not defined

In [8]:
from pymsa import MSA, Entropy, PercentageOfNonGaps, PercentageOfTotallyConservedColumns, Star, SumOfPairs
from pymsa import PAM250, Blosum62, FileMatrix
from pymsa.util.fasta import print_alignment


def run_all_scores(sequences: list) -> None:
    aligned_sequences = list(pair[1] for pair in sequences)
    sequences_id = list(pair[0] for pair in sequences)

    msa = MSA(aligned_sequences, sequences_id)
    print_alignment(msa)

    # Percentage of non-gaps and totally conserved columns
    non_gaps = PercentageOfNonGaps(msa)
    totally_conserved_columns = PercentageOfTotallyConservedColumns(msa)

    percentage = non_gaps.compute()
    print("Percentage of non-gaps: {0} %".format(percentage))

    conserved = totally_conserved_columns.compute()
    print("Percentage of totally conserved columns: {0}".format(conserved))

    # Entropy
    value = Entropy(msa).compute()
    print("Entropy score: {0}".format(value))

    # Sum of pairs
    value = SumOfPairs(msa, Blosum62()).compute()
    print("Sum of Pairs score (Blosum62): {0}".format(value))

    value = SumOfPairs(msa, PAM250()).compute()
    print("Sum of Pairs score (PAM250): {0}".format(value))

    value = SumOfPairs(msa, FileMatrix('PAM380.txt')).compute()
    print("Sum of Pairs score (PAM380): {0}".format(value))

    # Star
    value = Star(msa, Blosum62()).compute()
    print("Star score (Blosum62): {0}".format(value))

    value = Star(msa, PAM250()).compute()
    print("Star score (PAM250): {0}".format(value))


if __name__ == '__main__':
    sequences = [("1g41",
                  "S-EMTPREIVSELDQHIIGQADAKRAVAIALRNRWRRMQLQEPLRHE--------VTP-KNILMIGPTGVGKTEIARRLAKLANAPFIKVEATKFT----"
                  "VGKEVDSIIRDLTDSAMKLVRQQEIAKNR---------------------------------------------------------------------LI"
                  "DDEAAKLINPEELKQKAIDAVE--QNGIVFIDEIDKICKKGEYSGADVSREGVQRDLLPLVEGSTVSTKHGMVKTDHILFIASGAFQVARPSDL------"
                  "-----------IPELQGRLPIR-VEL---TALSAADFERILTEPHASLTEQYKALMATEGVNIAFTTDAVKKIAEAAFRVNEKTENIGARRLHTVMERLM"
                  "DKISFSASDMNGQTVNIDAAYVADALGEVVENEDLSRFIL"),
                 ("1e94",
                  "HSEMTPREIVSELDKHIIGQDNAKRSVAIALRNRWRRMQLNEELRHE--------VTP-KNILMIGPTGVGKTEIARRLAKLANAPFIKVEATKFTEVGY"
                  "VGKEVDSIIRDLTDAAVKMVRVQAIEKNRYRAEELAEERILDVLIPPAKNNWGQTEQQQEPSAARQAFRKKLREGQLDDKEIEKQKARKLKIKDAMKLLI"
                  "EEEAAKLVNPEELKQDAIDAVE--QHGIVFIDEIDKICKRGESSGPDVSREGVQRDLLPLVEGCTVSTKHGMVKTDHILFIASGAFQIAKPSDL------"
                  "-----------IPELQGRLPIR-VEL---QALTTSDFERILTEPNASITVQYKALMATEGVNIEFTDSGIKRIAEAAWQVNESTENIGARRLHTVLERLM"
                  "EEISYDASDLSGQNITIDADYVSKHLDALVADEDLSRFIL"),
                 ("1e32",
                  "R-ED-EEESLNEVGYDDVGG--CRKQLAQ-----I-KEMVELPLRHPALFKAIGVKPP-RGILLYGPPGTGKTLIARAVANETGAFFFLINGPEIM-SKL"
                  "A-GESESN--------------------------------------------------------------------------------------------"
                  "-------------LRKAFEEAEKNAPAIIFIDELDAIAPKREKTHGEVERRIVSQ-LLTLMDGL--------KQRAHVIVMAATN----RPNSIDPALRR"
                  "FGRFDREVDIGIPDATGRLEILQIHTKNMKLADDVDLEQVANETHGH---------------------------------------VGADLAALCSEAAL"
                  "QAIRKKMDLIDLEDETIDAEVM-NSL-AVTMDDFRWALSQ"),
                 ("1d2n",
                  "------EDYASYIMNGIIKWGDP---VTRVLD--DGELLVQQTKNSD--------RTPLVSVLLEGPPHSGKTALAAKIAEESNFPFIKICSPDKM-IGF"
                  "SETAKCQA--------------------------------------------------------------------------------------------"
                  "-------------MKKIFDDAYKSQLSCVVVDDIERLLDYV-PIGPRFSNLVLQA-LLVLLKKA-------PPQGRKLLIIGTTS----R-KDVLQEMEM"
                  "LNA---------------------------------FSTTIHVPNIATGEQL--LEALEL-LGNFKDKE---RTTIAQQVKGKKVWIGIKKLLMLIEM--"
                  "-------------SLQMDPEYRVRKFLALLREEGAS-PLD")]
    #run_all_scores(sequences)
    run_all_scores(list_of_pairs)

NameError: name 'list_of_pairs' is not defined

In [4]:
from pymsa.core.msa import MSA


def read_fasta_file_as_list_of_pairs(proaln_fasta: str) -> list:
    """
    Read a file in FASTA format as list of pairs (sequence id, sequence).

    :param file_name: FASTA file.
    :return: List of pairs.
    """
    list_of_pairs = []
    key = ''
    value = ''

    with open(proaln_fasta, 'r') as file:
        for line in file:
            if line[0] == '>':
                if key != '':
                    list_of_pairs.append((key, value))
                key = line[1:].rstrip()
                value = ''
            else:
                value += line.rstrip()

    list_of_pairs.append((key, value))
    return list_of_pairs

In [6]:
read_fasta_file_as_list_of_pairs(proaln_fasta)

NameError: name 'proaln_fasta' is not defined

In [211]:
#This didn't work but may not need it
#from Bio.Align import AlignInfo
#summary_align = AlignInfo.SummaryInfo(alignment)
#consensus = summary_align.replacement_dictionary()
#print(consensus)

In [20]:
#This wasn't that informative but save for now
substitutions = alignment2.substitutions
print(substitutions)

        A        C       G       T
A 94918.0     36.0   230.0    12.0
C    36.0 107062.0     0.0  1658.0
G   230.0      0.0 45574.0     0.0
T    12.0   1658.0     0.0 90502.0



### Iterate through alignments and finds sequences containing single or double gaps that are reported to the user. 

In [22]:
single_gap_id = []
double_gap_id = []
for record in alignment2:
#for record in nuc_align_fsa:
    loc = []
    seqid = record.id
    sequence = str(record.seq)
    #regex_single = r'[A-Z]+-[A-Z]'
    #regex_double = r'[A-Z]+--[A-Z]'
    regex_single = re.compile(r'[A-Z]+-[A-Z]')
    regex_double = re.compile(r'[A-Z]+--[A-Z]')
    
    single_gap = re.search(regex_single, sequence)
    if single_gap is not None:
        loc = single_gap.end()
        print('single gap found in', record.id, 'at location', loc)
        single_gap_id.append(seqid)
    
    double_gap = re.search(regex_double, sequence)
    if double_gap is not None:
        loc = double_gap.end()
        print('double gap found in', record.id, 'at location', loc)
        double_gap_id.append(seqid)


single gap found in OP894092 at location 1131
double gap found in OP894093 at location 1089


### Iterates though alignments, finds gaps, prints list of seqids and replaces single or double gap with N. Repeat of above but may not be the best solution REQUIRES STRING OUTPUT SKIPPING FOR NOW

In [208]:
single_gap_id = []
double_gap_id = []
for record in alignment2:
    seqid = record.id
    sequence = str(record.seq)
    regex_single = r'[A-Z]+-[A-Z]'
    regex_double = r'[A-Z]+--[A-Z]'
    regex_single_replace = re.compile("[-]")
    regex_double_replace = re.compile("[--]")
    
    single_gap = re.search(regex_single, sequence)
    if single_gap:
        print('single gap found', record.id)
        single_gap_id.append(seqid)
        sequence = regex_single_replace.sub("N", sequence)
        record.seq = sequence
        print(record.id, record.seq)
        
    
    double_gap = re.search(regex_double, sequence)
    if double_gap:
        print('double gap found', record.id)
        double_gap_id.append(seqid)
        sequence = regex_double_replace.sub("NN", sequence)
        record.seq = sequence
        print(record.id, record.seq)

single gap found OP894092
OP894092 ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATCATCAATGACTCATTTATTGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTGGCTCCCTATTAGGCGTATGCTTAGCAATCCAAATCCTAACAGGCCTATTCCTAGCAATACATTACACATCTGACACCGCTACCGCCTTCTACTCCGTCACCCATATCTGCCGAGACGTCAACTACGGCTGAGTTCTGCGCTATCTCCACGCCAACGGAGCATCCATATTCTTCATCTGCCTATTCCTACATGTAGGCCGAGGCATCTATTATGGTTCCTACACATATACAGAAACATGAAACATCGGAATCATCCTTCTCTTTGCCGTTATAGCAACAGCCTTCATAGGATATGTCCTCCCATGAGGACAAATATCTTTCTGAGGCGCAACAGTCATCACCAATCTCCTCTCAGCTATCCCCTATATCGGAACTAATCTCGTAGAGTGAGTTTGAGGGGGCTTCTCCGTAGACAAAGCTACCCTCACTCGATTCTTCGCCCTCCACTTTCTCCTCCCATTTATTATCGCAGCCCTAGTAATAGTACATCTGCTCTTCCTACATGAAACAGGCTCAAACAACCCCACAGGAATACCATCAGATGTAGACATAATCCCCTTCCACCCTTACTACACAATTAAAGATATTCTAGGACTAGTCCTCATACTTATAGTTCTATTATCACTAGTGCTATTCTCACCAGACCTCCTAGGAGACCCAGACAACTACACCCCAGCCAACCCGCTCAACACCCCACCCCATATTAAGCCAGAATGATACTTCCTATTTGCCTATGCCATCCTACGATCAATCCCAAATAAACTAGGAGGAGTTGTAGCCCTAGTCCTTTCCATTCTTATCCTAGCCGTAATTCCCCTACTTCACACATCAAAACAACGCAGCATAACCTTCCGACCTATCAGCCA

In [257]:
#This works but as string and would require the output to be written as a string. SeqIO.write will not work here
#Save for now may be useful if want to do global replacements
sequences = []
for index, seq_record in enumerate(SeqIO.parse(nuc_align_fsa, "fasta")):
    s = seq_record
    if seq_record.id in single_gap_id:
        sequence = str(seq_record.seq) 
        regex_single_replace = re.compile("[-]")
        sequence = regex_double_replace.sub("N", sequence)
        seq_record.seq = sequence
        sequences.append(s)
        print(seq_record.id, seq_record.seq)
    
    if seq_record.id in double_gap_id:
        sequence = str(seq_record.seq)   
        regex_double_replace = re.compile("[--]")
        sequence = regex_double_replace.sub("N", sequence)
        seq_record.seq = sequence
        sequences.append(s)
    else:
        sequences.append(s)
       
#SeqIO.write(sequences, "revisedfasta", "fasta")  

OP894092 ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATCATCAATGACTCATTTATTGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTGGCTCCCTATTAGGCGTATGCTTAGCAATCCAAATCCTAACAGGCCTATTCCTAGCAATACATTACACATCTGACACCGCTACCGCCTTCTACTCCGTCACCCATATCTGCCGAGACGTCAACTACGGCTGAGTTCTGCGCTATCTCCACGCCAACGGAGCATCCATATTCTTCATCTGCCTATTCCTACATGTAGGCCGAGGCATCTATTATGGTTCCTACACATATACAGAAACATGAAACATCGGAATCATCCTTCTCTTTGCCGTTATAGCAACAGCCTTCATAGGATATGTCCTCCCATGAGGACAAATATCTTTCTGAGGCGCAACAGTCATCACCAATCTCCTCTCAGCTATCCCCTATATCGGAACTAATCTCGTAGAGTGAGTTTGAGGGGGCTTCTCCGTAGACAAAGCTACCCTCACTCGATTCTTCGCCCTCCACTTTCTCCTCCCATTTATTATCGCAGCCCTAGTAATAGTACATCTGCTCTTCCTACATGAAACAGGCTCAAACAACCCCACAGGAATACCATCAGATGTAGACATAATCCCCTTCCACCCTTACTACACAATTAAAGATATTCTAGGACTAGTCCTCATACTTATAGTTCTATTATCACTAGTGCTATTCTCACCAGACCTCCTAGGAGACCCAGACAACTACACCCCAGCCAACCCGCTCAACACCCCACCCCATATTAAGCCAGAATGATACTTCCTATTTGCCTATGCCATCCTACGATCAATCCCAAATAAACTAGGAGGAGTTGTAGCCCTAGTCCTTTCCATTCTTATCCTAGCCGTAATTCCCCTACTTCACACATCAAAACAACGCAGCATAACCTTCCGACCTATCAGCCAATGCCTATTCTGACTTCTAGTTGCTG

### Replace gap characters with Ns, retranslate and save new nucleotide and protein files

In [23]:
#This works and does everything as a seq_record within biopython
from Bio.Seq import MutableSeq
sequences = []
protein_seq = []
for index, seq_record in enumerate(SeqIO.parse(nuc_align_fsa, "fasta")):
    s = seq_record
    if s.id in single_gap_id:
        sequence = MutableSeq(s.seq)
        location = sequence.index("-")
        sequence[location] = 'N'
        s.seq = sequence
        sequences.append(s)
        #print(seq_record.id, seq_record.seq)
    
    elif s.id in double_gap_id:
        sequence = MutableSeq(s.seq)  
        location = sequence.index("-")
        sequence[location] = 'N'
        sequence[location + 1] = 'N'
        s.seq = sequence
        sequences.append(s)
        #print(seq_record.id, seq_record.seq)
    else:
        sequences.append(s)
SeqIO.write(sequences, "revisedfasta", "fasta")  


25

In [24]:
protein_seq = []
for seq_record in SeqIO.parse("revisedfasta", "fasta"):
    s = seq_record
    s.seq = s.seq.translate(table = translation_table, to_stop=True)
    protein_seq.append(s)
#print(protein_seq)
SeqIO.write(protein_seq, "revised_protein_trans", "fasta")

25

## Repeat alignments and score

In [25]:
in_file = input_file
out_file = nuc_alignment
clustalw_cline = ClustalwCommandline("clustalo", infile="revisedfasta", outfile="revisedfasta.aln")
print(clustalw_cline)

clustalo -infile=revisedfasta -outfile=revisedfasta.aln


In [26]:
clustalw_cline()

('Using 8 threads\nRead 25 sequences (type: DNA) from revisedfasta\nnot more sequences (25) than cluster-size (100), turn off mBed\nCalculating pairwise ktuple-distances...\nKtuple-distance calculation progress: 0 % (0 out of 325)\nKtuple-distance calculation progress: 1 % (4 out of 325)\nKtuple-distance calculation progress: 28 % (93 out of 325)\nKtuple-distance calculation progress: 35 % (116 out of 325)\nKtuple-distance calculation progress: 42 % (137 out of 325)\nKtuple-distance calculation progress: 43 % (143 out of 325)\nKtuple-distance calculation progress: 47 % (154 out of 325)\nKtuple-distance calculation progress: 54 % (176 out of 325)\nKtuple-distance calculation progress: 55 % (181 out of 325)\nKtuple-distance calculation progress: 59 % (192 out of 325)\nKtuple-distance calculation progress: 60 % (195 out of 325)\nKtuple-distance calculation progress: 64 % (209 out of 325)\nKtuple-distance calculation progress: 66 % (217 out of 325)\nKtuple-distance calculation progress: 67

In [27]:
in_file = input_file
out_file = nuc_alignment
clustalw_cline = ClustalwCommandline("clustalo", infile="revised_protein_trans", outfile="revised_protein_trans.aln")
print(clustalw_cline)

clustalo -infile=revised_protein_trans -outfile=revised_protein_trans.aln


In [28]:
clustalw_cline()

('Using 8 threads\nRead 25 sequences (type: Protein) from revised_protein_trans\nnot more sequences (25) than cluster-size (100), turn off mBed\nCalculating pairwise ktuple-distances...\nKtuple-distance calculation progress: 0 % (0 out of 325)\nKtuple-distance calculation progress: 1 % (4 out of 325)\nKtuple-distance calculation progress: 19 % (63 out of 325)\nKtuple-distance calculation progress: 21 % (70 out of 325)\nKtuple-distance calculation progress: 24 % (80 out of 325)\nKtuple-distance calculation progress: 26 % (86 out of 325)\nKtuple-distance calculation progress: 31 % (102 out of 325)\nKtuple-distance calculation progress: 32 % (106 out of 325)\nKtuple-distance calculation progress: 38 % (126 out of 325)\nKtuple-distance calculation progress: 45 % (147 out of 325)\nKtuple-distance calculation progress: 49 % (162 out of 325)\nKtuple-distance calculation progress: 50 % (163 out of 325)\nKtuple-distance calculation progress: 54 % (176 out of 325)\nKtuple-distance calculation pr

In [29]:
alignment4 = AlignIO.read("revisedfasta.aln", "clustal")
SeqIO.write(alignment4, "revisedfasta.alnfsa", "fasta")

25

In [30]:
file_name = "revisedfasta.alnfsa"
read_fasta_file_as_list_of_pairs(file_name)


[('OP894092',
  'ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATCATCAATGACTCATTTATTGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTGGCTCCCTATTAGGCGTATGCTTAGCAATCCAAATCCTAACAGGCCTATTCCTAGCAATACATTACACATCTGACACCGCTACCGCCTTCTACTCCGTCACCCATATCTGCCGAGACGTCAACTACGGCTGAGTTCTGCGCTATCTCCACGCCAACGGAGCATCCATATTCTTCATCTGCCTATTCCTACATGTAGGCCGAGGCATCTATTATGGTTCCTACACATATACAGAAACATGAAACATCGGAATCATCCTTCTCTTTGCCGTTATAGCAACAGCCTTCATAGGATATGTCCTCCCATGAGGACAAATATCTTTCTGAGGCGCAACAGTCATCACCAATCTCCTCTCAGCTATCCCCTATATCGGAACTAATCTCGTAGAGTGAGTTTGAGGGGGCTTCTCCGTAGACAAAGCTACCCTCACTCGATTCTTCGCCCTCCACTTTCTCCTCCCATTTATTATCGCAGCCCTAGTAATAGTACATCTGCTCTTCCTACATGAAACAGGCTCAAACAACCCCACAGGAATACCATCAGATGTAGACATAATCCCCTTCCACCCTTACTACACAATTAAAGATATTCTAGGACTAGTCCTCATACTTATAGTTCTATTATCACTAGTGCTATTCTCACCAGACCTCCTAGGAGACCCAGACAACTACACCCCAGCCAACCCGCTCAACACCCCACCCCATATTAAGCCAGAATGATACTTCCTATTTGCCTATGCCATCCTACGATCAATCCCAAATAAACTAGGAGGAGTTGTAGCCCTAGTCCTTTCCATTCTTATCCTAGCCGTAATTCCCCTACTTCACACATCAAAACAACGCAGCATAACCTTCCGACCTATCAGCCAATGCCTATTCTGACTTCT

In [32]:
run_all_scores(list_of_pairs)

OP894092	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATCATCAATGACTCATTTATTGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894093	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATCATCAATGACTCATTTATTGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894094	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATCATCAATGACTCATTTATTGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894095	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATCATCAATGACTCATTTATTGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894096	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATCATCAATGACTCATTTATTGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894097	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATTATCAATGACTCATTCATCGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894098	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATTATCAATGACTCATTCATCGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894099	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATTATCAATGACTCATTCATCGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894100	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATTATCAATGACTCATTCATCGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894101	A

OP894102	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATTATCAATGACTCATTCATCGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894103	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATTATCAATGACTCATTCATCGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894104	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATTATCAATGACTCATTCATCGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894105	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATTATCAATGACTCATTCATCGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894106	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATTATCAATGACTCATTCATCGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894107	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATTATCAATGACTCATTCATCGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894108	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATTATCAATGACTCATTCATCGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894109	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATCATCAATGACTCATTTATTGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894110	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATCATCAATGACTCATTTATTGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894111	A

OP894112	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATCATCAATGACTCATTTATTGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894113	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATCATCAATGACTCATTTATTGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894114	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATCATCAATGACTCATTTATTGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894115	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATCATCAATGACTCATTTATTGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG
OP894116	ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATCATCAATGACTCATTTATTGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTG

OP894092	GCTCCCTATTAGGCGTATGCTTAGCAATCCAAATCCTAACAGGCCTATTCCTAGCAATACATTACACATCTGACACCGCTACCGCCTTCTACTCCGTCAC
OP894093	GCTCCCTATTAGGCGTATGCTTAGCAATCCAAATCCTAACAGGCCTATTCCTAGCAATACATTACACATCTGACACCGCTACCGCCTTCTACTCCGTCAC
OP894094	GCTCCCTATTAGGCGTATGCTTAGCAATCCAAATCCTAACAGGCCTATTCCTAGCAATACATTACACATCTGACACCGCTACCGCCTTCTACTCCGTCAC
OP894095	GCTCCCTATTAGGCGTATGCTTAGCAATCCAAATCCTAACAGGCCTATTCCTAGCAATACATTACACATCTGACACCGCTACCGCCTTCTACTCCGTCAC
OP894096	

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



CCATATCTGCCGAGACGTCAACTACGGCTGAGTTCTGCGCTATCTCCACGCCAACGGAGCATCCATGTTCTTCATCTGCCTATTCCTACATGTAGGCCGA
OP894100	CCATATCTGCCGAGACGTCAACTACGGCTGAGTTCTGCGCTATCTCCACGCCAACGGAGCATCCATATTCTTCATCTGCCTATTCCTACATGTAGGCCGA
OP894101	CCATATCTGCCGAGACGTCAACTACGGCTGAGTTCTGCGCTATCTCCACGCCAACGGAGCATCCATATTCTTCATCTGCCTATTCCTACATGTAGGCCGA
OP894102	CCATATCTGCCGAGACGTCAACTACGGCTGAGTTCTGCGCTATCTCCACGCCAACGGAGCATCCATATTCTTCATCTGCCTATTCCTACATGTAGGCCGA
OP894103	CCATATCTGCCGAGACGTCAACTACGGCTGAGTTCTGCGCTATCTCCACGCCAACGGAGCATCCATATTCTTCATCTGCCTATTCCTACATGTAGGCCGA
OP894104	CCATATCTGCCGAGACGTCAACTACGGCTGAGTTCTGCGCTATCTCCACGCCAACGGAGCATCCATGTTCTTCATCTGCCTATTCCTACATGTAGGCCGA
OP894105	CCATATCTGCCGAGACGTCAACTACGGCTGAGTTCTGCGCTATCTCCACGCCAACGGAGCATCCATGTTCTTCATCTGCCTATTCCTACATGTAGGCCGA
OP894106	CCATATCTGCCGAGACGTCAACTACGGCTGAGTTCTGCGCTATCTCCACGCCAACGGAGCATCCATGTTCTTCATCTGCCTATTCCTACATGTAGGCCGA
OP894107	CCATATCTGCCGAGACGTCAACTACGGCTGAGTTCTGCGCTATCTCCACGCCAACGGAGCATCCATGTTCTTCATCTGCCTATTCCTACATGTAGGCCGA
OP894108	CCATATCTGC

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



ACCCCAGCCAACCCGCTCAACACCCCACCCCA

OP894092	TATTAAGCCAGAATGATACTTCCTATTTGCCTATGCCATCCTACGATCAATCCCAAATAAACTAGGAGGAGTTGTAGCCCTAGTCCTTTCCATTCTTATC
OP894093	TATTAAGCCAGAATGATACTTCCTATTTGCCTATGCCATCCTACGATCAATCCCAAATAAACTAGGAGGAGTTGTAGCCCTAGTCCTTTCCATTCTTATC
OP894094	TATTAAGCCAGAAGGAGTTGTAGCCCTAGTCCTTTCCATCCTTATC
OP894098	TATTAAGCCAGAATGATATTTCCTATTTGCCTATGCCATCCTACGATCAATCCCAAATAAATTAGGAGGAGTTGTAGCCCTAGTCCTTTCCATCCTTATC
OP894099	TATTAAGCCAGAATGGTATTTCCTATTTGCCTATGCCATCCTACGGTCAATCCCAAATAAATTAGGAGGAGTTGTAGCCCTAGTCCTTTCCATCCTTATC
OP894100	TATTAAGCCAGAATGATATTTCCTATTTGCCTATGCCATCCTACGATCAATCCCAAATAAATTAGGAGGAGTTGTAGCCCTAGTCCTTTCCATCCTTATC
OP894101	TATTAAGCCAGAATGATATTTCCTATTTGCCTATGCCATCCTACGATCAATCCCAAATAAATTAGGAGGAGTTGTAGCCCTAGTCCTTTCCATCCTTATC
OP894102	TATTAAGCCAGAATGATATTTCCTATTTGCCTATGCCATCCTACGATCAATCCCAAATAAATTAGGAGGAGTTGTAGCCCTAGTCCTTTCCATCCTTATC
OP894103	TATTAAGCCAGAATGATATTTCCTATTTGCCTATGCCATCCTACGATCAATCCCAAATAAATTAGGAGGAGTCGTAGCCCTAGTCCTTTCCATCCTTATC
OP894104	TATTAAGCCAGAATGATATTT

OP894105	TATTAAGCCAGAATGATATTTCCTATTTGCCTATGCCATCCTACGGTCAATCCCAAATAAATTAGGAGGAGTTGTAGCCCTAGTCCTTTCCATCCTTATC
OP894106	TATTAAGCCAGAATGATATTTCCTATTTGCCTATGCCATCCTACGGTCAATCCCAAATAAATTAGGAGGAGTTGTAGCCCTAGTCCTTTCCATCCTTATC
OP894107	TATTAAGCCAGAATGATATTTCCTATTTGCCTATGCCATCCTACGGTCAATCCCAAACATTCTTATC
OP894114	TATTAAGCCAGAATGATACTTCCTATTTGCCTATGCCATCCTACGATCAATCCCAAATAAACTAGGAGGAGTTGTAGCCCTAGTCCTTTCCATTCTTATC
OP894115	TATTAAGCCAGAATGATACTTCCTATTTGCCTATGCCATCCTACGATCAATCCCAAATAAACTAGGAGGAGTTGTAGCCCTAGTCCTTTCCATTCTTATC
OP894116	TATTAAGCCAGAATGATACTTCCTATTTGCCTATGCCATCCTACGATCAATCCCAAATAAACTAGGAGGAGTTGTAGCCCTAGTCCTTTCCATTCTTATC

OP894092	CTAGCCGTAATTCCCCTACTTCACACATCAAAACAACGCAGCATAACCTTCCGACCTATCAGCCAATGCCTATTCTGACTTCTAGTTGCTGATCTCCTCA
OP894093	CTAGCCGTAATTCCCCTACTTCACACATCAAAACAACGCAGCATAACCTTCCGACCTATCAGCCAATGCCTATTCTGACTTCTAGTTGCTGATCTCCTCA
OP894094	CTAGCCGTAATTCCCCTACTTCACACATCAAAACAACGCAGCATAACCTTCCGACCTATCAGCCAATGCCTATTCTGACTTCTAGTTGCTGATCTCCTCA
OP894095	CTAGCCGTAATTCCCCTACTTCACACATCAAAA

OP894096	CTAGCCGTAATTCCCCTACTTCACACATCAAAACAACGCAGCATAACCTTCCGACCTATCAGCCAATGCCTATTCTGACTTCTAGTTGCTGATCTCCTCA
OP894097	CTAGCCGTAATTCCCCTACTTCACACATCAAAACAACGCAGCATAACCTTCCGACCTATCAGCCAATGCCTATTCTGACTTCTAGTTGCTGATCTCCTCA
OP894098	CTAGCCGTAATTCCCCTACTTCACACATCAAAACAACGCAGCATAACCTTCCGACCTATCAGCCAATGCCTATTCTGACCTTCCGACCTATCAGCCAATGCCTATTCTGACTTCTAGTTGCTGATCTCCTCA
OP894100	CTAGCCGTAATTCCCCTACTTCACACATCAAAACAACGCAGCATAACCTTCCGACCTATCAGCCAATGCCTATTCTGACTTCTAGTTGCTGACCTCCTCA
OP894101	CTAGCCGTAATTCCCCTACTTCACACATCAAAACAACGCAGCATAACCTTCCGACCTATCAGCCAATGCCTATTCTGACTTCTAGTTGCTGATCTCCTCA
OP894102	CTAGCCGTAATTCCCCTACTTCACACATCAAAACAACGCAGCATAACCTTCCGACCTATCAGCCAATGCCTATTCTGACTTCTAGTTGCTGATCTCCTCA
OP894103	CTAGCCGTAATTCCCCTACTTCACACATCAAAACAACGCAGCATAACCTTCCGACCTATCAGCCAATGCCTATTCTGACTTCTAGTTGCTGATCTCCTCA
OP894104	CTAGCCGTAATTCCCCTACTTCACACATCAAAACAACGCAGCATAACCTTCCGACCTATCAGCCAATGCCTATTCTGACTTCTAGTTGCTGATCTCCTCA
OP894105	CTAGCCGTAATTCCCCTACTTCACACATCAAAACAACGCAGCATAACCTTCCGACCTATCAGCCAATGCCTATTCTGAC

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Test blocks

In [251]:
#testing options
from Bio.Seq import MutableSeq
sequences = []
for index, seq_record in enumerate(SeqIO.parse(nuc_align_fsa, "fasta")):
    for id in double_gap_id:
        if id in seq_record.id:
            sequence = MutableSeq(seq_record.seq)
            location = sequence.index("-")
            print(location)
            sequence[location] = 'N'
            sequence[location + 1] = 'N'
            #regex_double_replace = re.compile("[--]")
            #sequence = regex_double_replace.sub("N", sequence)
            seq_record.seq = sequence
            print(seq_record.id, seq_record.seq)
SeqIO.write(seq_record, "revisedfasta", "fasta")  

1086
OP894093 ATGACCAACATCCGAAAATCCCACCCACTATTCAAAATCATCAATGACTCATTTATTGACCTTCCCGCCCCCTCAAGTATCTCATCCTGATGAAATTTTGGCTCCCTATTAGGCGTATGCTTAGCAATCCAAATCCTAACAGGCCTATTCCTAGCAATACATTACACATCTGACACCGCTACCGCCTTCTACTCCGTCACCCATATCTGCCGAGACGTCAACTACGGCTGAATTCTGCGCTATCTCCACGCCAACGGAGCATCCATATTCTTCATCTGCCTATTCCTACATGTAGGCCGAGGCATCTATTATGGTTCCTACACATATACAGAAACATGAAACATCGGAATCATCCTTCTCTTTGCCGTTATAGCAACAGCCTTCATAGGATATGTCCTCCCATGAGGACAAATATCTTTCTGAGGCGCAACAGTCATCACCAATCTCCTCTCAGCTATCCCCTATATCGGAACTAATCTCGTAGAGTGAGTTTGAGGGGGCTTCTCCGTAGACAAAGCTACCCTCACTCGATTCTTCGCCCTCCACTTTCTCCTCCCATTTATTATCGCAGCCCTAGTAATAGTACATCTGCTCTTCCTACATGAAACAGGCTCAAACAACCCCACAGGAATACCATCAGATGTAGACATAATCCCCTTCCACCCTTACTACACAATTAAAGATATTCTAGGACTAGTCCTCATACTTATAGTTCTATTATCACTAGTGCTATTCTCACCAGACCTCCTAGGAGACCCAGACAACTACACCCCAGCCAACCCGCTCAACACCCCACCCCATATTAAGCCAGAATGATACTTCCTATTTGCCTATGCCATCCTACGATCAATCCCAAATAAACTAGGAGGAGTTGTAGCCCTAGTCCTTTCCATTCTTATCCTAGCCGTAATTCCCCTACTTCACACATCAAAACAACGCAGCATAACCTTCCGACCTATCAGCCAATGCCTATTCTGACTTCTAGT

1